### create external location & unity catalog 
### AUTO CDC API's replace the Apply changes API's
### remove live its optional
### Load the data to Landing zone and move it to Bronze layer

### Hive Metastore 
### create mount point
### in case of Azure , create a adls account and container landing and directoreis orders & customers
#### in databricks I have craeted a valume and upload data 
### A table can belongs to one pipeline only. otherwise it will create conflict in case of full refresh

In [0]:

create streaming table if not exists orders_bronze_lakeflow_declaraive
as
select *,
current_timestamp() as load_time,
_metadata.file_name as file_name
from stream(cloud_files('/Volumes/workspace/default/dlt_volume/input/orders', 'csv', map('cloud_files.inferColumnsTypes', 'true')));

create streaming table if not exists customers_bronze_lakeflow_declaraive
as
select *, 
current_timestamp() as load_time,
_metadata.file_name as file_name
from stream(cloud_files('/Volumes/workspace/default/dlt_volume/input/customers', 'csv', map('cloud_files.inferColumnTypes', 'true')));




### Step 2 : Take the data from Bronze and clean it and load it in the silver_cleaned

In [0]:
create streaming table if not exists orders_cleaned_silver_lakeflow_declaraive(
constraint valid_orders expect(order_id is not null) on violation drop row,
constraint valid_customers expect(customer_id is not null) on violation drop row
)
as
select OrderID as order_id,
OrderDate as order_date,
CustomerID as customer_id,
TotalAmount as total_amount,
Status as order_status,
file_name,
load_time
from 
stream(orders_bronze);

create streaming table if not exists customers_cleaned_silver_lakeflow_declaraive(
constraint valid_customers expect(customer_id is not null) on violation drop row
)
as
select CustomerID as customer_id,
CustomerName as customer_name,
ContactNumber as contact_number,
Email as email,
Address as city,
DateOfBirth as date_of_birth,
RegistrationDate as resistration_date,
file_name,
load_time
from 
stream(customers_bronze);



### take the data from cleaned silver and handle duplicates , scd 2 and load it to silver 

In [0]:
create streaming table orders_silver_lakeflow_declaraive;
create flow orders_silver_flow as auto cdc
into orders_silver_lakeflow_declaraive
from stream(orders_cleaned_silver_lakeflow_declaraive)
keys(order_id)
sequence by load_time;

create streaming table customer_silver_lakeflow_declaraive;
create flow customers_silver_flow as auto cdc
into customer_silver_lakeflow_declaraive
from stream(customers_cleaned_silver_lakeflow_declaraive)
keys(customer_id)
sequence by load_time
stored as scd type 2;

### Gold Layer : Take the date from silver layer and perform aggregations and create Gold layer

In [0]:
create materialized view if not exists city_wise_sales_gold_lakeflow_declaraive
as
select city , sum(total_amount) as total_sales
from orders_silver_lakeflow_declaraive o
join customer_silver_lakeflow_declaraive c
on o.customer_id = c.customer_id
group by city;

### No upload new orders & customers files and you will see for customers(customer_id 80) it will perfomr scd type 2 and for orders (order_id 400) it will perform merge.
### folders will be created for bronze , silver & Gold. 
### for Gold folder : Enzyme folder is also cretaed to prevent full load computation when incremental data arrives.
## select * from orders_silver where order_id=400; > merge
## select * from customer_silver where customer_id=80;  > scd type 2